In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm

# Paths
data_dir = "../FBMM/test"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")
model_save_path = "./models/optimized_efficientnet_b2_emotion_model.h5"
checkpoint_path = "./checkpoints/checkpoint_epoch_{epoch:02d}.h5"

# Configuration
batch_size = 64
num_epochs = 25
initial_lr = 1e-3
num_classes = 7
img_height, img_width = 260, 260
seed = 42  # For reproducibility

# Emotion categories
emotion_classes = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Data Preparation
def prepare_data_generators(train_dir, val_dir, test_dir, batch_size, img_height, img_width):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
    )

    val_test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='categorical',
        shuffle=True,
        seed=seed
    )

    val_generator = val_test_datagen.flow_from_directory(
        val_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='categorical',
        shuffle=False,
        seed=seed
    )

    test_generator = val_test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, val_generator, test_generator


# Compute class weights
def compute_weights(train_generator, num_classes):
    print("Computing class weights...")
    labels = train_generator.classes
    class_weights = compute_class_weight('balanced', classes=np.arange(num_classes), y=labels)
    return {i: float(weight) for i, weight in enumerate(class_weights)}


# Load the model and freeze initial layers
def load_model(num_classes):
    print("Loading and configuring the model...")
    base_model = tf.keras.applications.EfficientNetB2(
        input_shape=(img_height, img_width, 3),
        include_top=False,
        weights='imagenet'
    )

    # Freeze initial layers
    for layer in base_model.layers[:int(len(base_model.layers) * 0.8)]:
        layer.trainable = False

    # Build the model
    model = models.Sequential([
        layers.InputLayer(input_shape=(img_height, img_width, 3)),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.6),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model


# Dynamic Learning Rate Adjustment Callback
class DynamicLearningRate(tf.keras.callbacks.Callback):
    def __init__(self, initial_lr, patience=3, factor=0.5):
        super(DynamicLearningRate, self).__init__()
        self.initial_lr = initial_lr
        self.patience = patience
        self.factor = factor
        self.best_val_loss = np.Inf
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')
        if current_val_loss < self.best_val_loss:
            self.best_val_loss = current_val_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                new_lr = self.model.optimizer.learning_rate * self.factor
                tf.keras.backend.set_value(self.model.optimizer.learning_rate, new_lr)
                print(f"\nEpoch {epoch + 1}: Reducing learning rate to {new_lr:.6f}.")
                self.wait = 0


# Illusion Effect Callback
class IllusionCallback(tf.keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print(
            f"\rBatch {batch + 1} - Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}", 
            end=""
        )


# Prepare data generators
train_generator, val_generator, test_generator = prepare_data_generators(
    train_dir, val_dir, test_dir, batch_size, img_height, img_width
)


# Compute class weights
class_weights = compute_weights(train_generator, num_classes)


# Load model
model = load_model(num_classes)


# Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=initial_lr),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# Callbacks
checkpoint_cb = callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

early_stopping_cb = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

dynamic_lr_cb = DynamicLearningRate(initial_lr=initial_lr, patience=3, factor=0.5)
illusion_cb = IllusionCallback()


# Training
history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    class_weight=class_weights,
    callbacks=[checkpoint_cb, early_stopping_cb, dynamic_lr_cb, illusion_cb],
    verbose=1
)

# Evaluate on test data
print("\nEvaluating on test data...")
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_acc:.4f}")


Found 140 images belonging to 7 classes.
Found 140 images belonging to 7 classes.
Found 140 images belonging to 7 classes.


Computing class weights...
Loading and configuring the model...
Epoch 1/25
3/3 [==============================] - ETA: 0s - loss: 1.9832 - accuracy: 0.1429 
Epoch 1: val_loss improved from inf to 1.99626, saving model to ./checkpoints\checkpoint_epoch_01.h5


TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.